In [7]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from six.moves import cPickle as pickle
# from six.moves import range

pickle_file = './model/SVHN_multi.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Test set', test_dataset.shape, test_labels.shape)

Test set (13068, 32, 32, 1) (13068, 6)


In [16]:
def LecunLCN(X, image_shape, threshold=1e-4, radius=7, use_divisor=True):
    """
    Local Contrast Normalization
    :param X: tf_train_dataset
    :param image_shape: [batch_size, image_size, image_size, num_channels]
    """
    # Get Gaussian filter
    filter_shape = (radius, radius, image_shape[3], 1)
    # For process visualisation 
#     plt.rcParams['figure.figsize'] = (45.0, 100.0)    
#     f, ax = plt.subplots(nrows= radius * radius + 1, ncols=1)
#     [i.axis('off') for i in ax]
    ax = None
       
    filters, filters_asarray = gaussian_filter(filter_shape, ax)
    
    X = tf.convert_to_tensor(X, dtype=tf.float32)
    # Compute the Guassian weighted average by means of convolution
    convout = tf.nn.conv2d(X, filters, [1,1,1,1], 'SAME')

    # Subtractive step
    mid = int(np.floor(filter_shape[1] / 2.))

    # Make filter dimension broadcastable and subtract
    centered_X = tf.subtract(X, convout)

    # Boolean marks whether or not to perform divisive step
    if use_divisor:
        # Note that the local variances can be computed by using the centered_X
        # tensor. If we convolve this with the mean filter, that should give us
        # the variance at each point. We simply take the square root to get our
        # denominator

        # Compute variances
        sum_sqr_XX = tf.nn.conv2d(tf.square(centered_X), filters, [1,1,1,1], 'SAME')

        # Take square root to get local standard deviation
        denom = tf.sqrt(sum_sqr_XX)

        per_img_mean = tf.reduce_mean(denom)
        divisor = tf.maximum(per_img_mean, denom)
        # Divisise step
        new_X = tf.truediv(centered_X, tf.maximum(divisor, threshold))
    else:
        new_X = centered_X

    return new_X


def gaussian_filter_(kernel_shape, ax):
    x = np.zeros(kernel_shape, dtype = float)
    mid = np.floor(kernel_shape[0] / 2.)
    
    for kernel_idx in range(0, kernel_shape[2]):
        for i in range(0, kernel_shape[0]):
            for j in range(0, kernel_shape[1]):
                x[i, j, kernel_idx, 0] = gauss(i - mid, j - mid)
    
    return tf.convert_to_tensor(x / np.sum(x), dtype=tf.float32)


def gaussian_filter(kernel_shape, ax):
    # The Gaussian filter of the desired size initialized to zero
    filter_ = np.zeros(kernel_shape, dtype = float)
    mid = np.floor(kernel_shape[0] / 2.)  # Middle of kernel of Gaussian filter
    
    for kernel_idx in range(0, kernel_shape[2]):
        for i in range(0, kernel_shape[0]):  # go on width of Gaussian weighting window
            for j in range(0, kernel_shape[1]):  # go on height of Gaussian weighting window
                filter_[i, j, kernel_idx, 0] = gauss(i - mid, j - mid)
                # For process visualisation 
#                 _ = [i.ravel() for i in filter_] # (32, 32, 1) to (32, 32)
#                 ax[kernel_shape[0] * i + j].set_title(
#                     f"mid: {mid} i: {i} j: {j} x: {i - mid} y: {j - mid}", loc='center'
#                 )
#                 ax[kernel_shape[0] * i + j].imshow([i.ravel() for i in filter_])
    
    filter_ = filter_ / np.sum(filter_)    
#     ax[kernel_shape[0] * kernel_shape[1]].set_title(f"finish", loc='center')
#     ax[kernel_shape[0] * kernel_shape[1]].imshow([i.ravel() for i in filter_])
    return tf.convert_to_tensor(filter_, dtype=tf.float32), filter_


def gauss(x, y, sigma=3.0):
    Z = 2 * np.pi * sigma ** 2
    return  1. / Z * np.exp(-(x ** 2 + y ** 2) / (2. * sigma ** 2))

In [20]:
filename = "1.png"
fullname = os.path.join('./data/test', filename)
im = Image.open(fullname)
house_num = ''
image_index, _ = filename.split(".")
image_index = int(image_index)

# image_index = np.where(test
# for k in np.arange(test_labels[j,0]):
#     house_num += str(int(test_labels[j, int(k)+1]))
# print(house_num)

In [37]:
image_size = 32
num_labels = 11 # 0-9, + blank 
num_channels = 1 # grayscale

patch_size = 5
depth1 = 16
depth2 = 32
depth3 = 64
num_hidden1 = 64

graph = tf.Graph()
with graph.as_default():
    
    # Input data.
    tf_test_dataset = tf.placeholder(tf.float32, shape=(1, 32, 32, 1))
  
    # Variables.
    layer1_weights = tf.get_variable("W1", 
                                       shape=[patch_size, patch_size, num_channels, depth1],
                                       initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer1_biases = tf.Variable(tf.constant(1.0, shape=[depth1]), name='B1')
    layer2_weights = tf.get_variable("W2", 
                                       shape=[patch_size, patch_size, depth1, depth2],
                                       initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]), name='B2')
    layer3_weights = tf.get_variable("W3", 
                                       shape=[patch_size, patch_size, depth2, num_hidden1],
                                       initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden1]), name='B3')

    s1_w = tf.get_variable("WS1", shape=[num_hidden1, num_labels],
                             initializer=tf.contrib.layers.xavier_initializer())
    s1_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS1')
    s2_w = tf.get_variable("WS2", shape=[num_hidden1, num_labels], 
                             initializer=tf.contrib.layers.xavier_initializer())
    s2_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS2')
    s3_w = tf.get_variable("WS3", shape=[num_hidden1, num_labels],
                             initializer=tf.contrib.layers.xavier_initializer())
    s3_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS3')
    s4_w = tf.get_variable("WS4", shape=[num_hidden1, num_labels], 
                             initializer=tf.contrib.layers.xavier_initializer())
    s4_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS4')
    s5_w = tf.get_variable("WS5", shape=[num_hidden1, num_labels],
                            initializer=tf.contrib.layers.xavier_initializer())
    s5_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS5')
  
    # Model.
    def model(data, keep_prob, shape):
        LCN = LecunLCN(data, shape)
        conv = tf.nn.conv2d(LCN, layer1_weights, [1,1,1,1], 'VALID', name='C1')
        hidden = tf.nn.relu(conv + layer1_biases)
        lrn = tf.nn.local_response_normalization(hidden)
        sub = tf.nn.max_pool(lrn, [1,2,2,1], [1,2,2,1], 'SAME', name='S2')
        conv = tf.nn.conv2d(sub, layer2_weights, [1,1,1,1], padding='VALID', name='C3')
        hidden = tf.nn.relu(conv + layer2_biases)
        lrn = tf.nn.local_response_normalization(hidden)
        sub = tf.nn.max_pool(lrn, [1,2,2,1], [1,2,2,1], 'SAME', name='S4')
        conv = tf.nn.conv2d(sub, layer3_weights, [1,1,1,1], padding='VALID', name='C5')
        hidden = tf.nn.relu(conv + layer3_biases)
        hidden = tf.nn.dropout(hidden, keep_prob)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        
        logits1 = tf.matmul(reshape, s1_w) + s1_b
        logits2 = tf.matmul(reshape, s2_w) + s2_b
        logits3 = tf.matmul(reshape, s3_w) + s3_b
        logits4 = tf.matmul(reshape, s4_w) + s4_b
        logits5 = tf.matmul(reshape, s5_w) + s5_b
        
        return [logits1, logits2, logits3, logits4, logits5]
  
    # Training computation.
    [logits1, logits2, logits3, logits4, logits5] = model(tf_test_dataset, 1, [10, 32, 32, 1])
  
    predict = tf.stack([tf.nn.softmax(logits1), tf.nn.softmax(logits2),
                        tf.nn.softmax(logits3), tf.nn.softmax(logits4),
                        tf.nn.softmax(logits5)])  
  
    print(predict.shape)
    print(predict[0][0])
    test_prediction = tf.transpose(tf.argmax(predict, 2))
    saver = tf.train.Saver()
    print("test_dataset test_dataset", test_dataset[image_index])
#           np.expand_dims(test_dataset[image_index, :, :, :], axis=0).shape)
    a = np.expand_dims(test_dataset[image_index, :, :, :], axis=0)
#     print("a a ", a[0])

with tf.Session(graph=graph) as session:
    saver.restore(session, "./model/CNN_multi2.ckpt")
    print("Model restored.")  
    print('Initialized')
    
#     print(test_dataset[image_index, :, :, :]..shape)
    test_prediction = session.run(test_prediction, 
                                  feed_dict={tf_test_dataset : a,})
#                                   feed_dict={tf_test_dataset : test_dataset[image_index, :, :, :],})
    print(test_prediction)

(5, 1, 11)
Tensor("strided_slice_1:0", shape=(11,), dtype=float32)
test_dataset test_dataset [[[-1.2275152 ]
  [-1.2619097 ]
  [-1.3065858 ]
  ...
  [-1.6746991 ]
  [-1.7434882 ]
  [-1.7474095 ]]

 [[-0.61418915]
  [-0.6446623 ]
  [-0.6893384 ]
  ...
  [-0.9181777 ]
  [-1.2258744 ]
  [-1.4198927 ]]

 [[-0.33903292]
  [-0.3571572 ]
  [-0.3816417 ]
  ...
  [-0.5119618 ]
  [-0.7587122 ]
  [-0.8417042 ]]

 ...

 [[ 0.71839845]
  [ 0.93467563]
  [ 1.0339378 ]
  ...
  [ 0.7223198 ]
  [ 0.5847417 ]
  [ 0.4673552 ]]

 [[ 0.98963326]
  [ 1.2403051 ]
  [ 1.3636802 ]
  ...
  [ 1.2126422 ]
  [ 1.0789855 ]
  [ 0.93748593]]

 [[ 1.216192  ]
  [ 1.4466722 ]
  [ 1.580329  ]
  ...
  [ 1.3225574 ]
  [ 1.2193738 ]
  [ 1.0268381 ]]]
INFO:tensorflow:Restoring parameters from ./model/CNN_multi2.ckpt
Model restored.
Initialized


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node Conv2D (defined at <ipython-input-16-b129a1c28003>:19) ]]

Caused by op 'Conv2D', defined at:
  File "/home/worker/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/worker/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/worker/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/worker/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/worker/anaconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/worker/anaconda3/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 148, in star